# Minitarea 4

Por favor, en las respuestas de desarrollo expliquen lo que están haciendo. En las preguntas que son con desarrollo matemático pongan todos los pasos del cálculo (déjenlo bonito porfis :D).

Usen $\LaTeX$ para las fórmulas matemáticas.

En la parte de programación pueden usar lo que quieran, pero la auxiliar 3 les puede servir de *inspiración*.

# Hidden Markov Models (HMM), Maximum Entropy Markov Models (MEMM) and Conditional Random Field(CRF)

### Pregunta 1 (1 pt)
Para un problema de POS tagging se define el conjunto de etiquetas $S = \{ D, N, V, P \}$ y se tiene un Hidden Markov Model con los siguientes parámetros estimados a partir de un corpus de entrenamiento:

\begin{equation}
\begin{split}
q(D|N,P) &= 0.4 \\
q(D|w,P) &= 0 \qquad \forall w \in S, w \neq N \\
e(the|D) &= 0.6
\end{split}
\end{equation}

Luego para la oración: `Ella walks to the red house`, se tiene una tabla de programación dinámica con los siguientes valores:

\begin{equation}
\begin{split}
\pi(3,D,P)&=0.1\\
\pi(3,N,P)&=0.2\\
\pi(3,V,P)&=0.01\\
\pi(3,P,P)&=0.5
\end{split}
\end{equation}

Con esta información, calcule el valor de $\pi(4,P,D)$. Puede dejar el resultado expresado como una fracción.


**Respuesta**

La siguiente es la definición recursiva del Algoritmo de Viterbi:
$\pi(k,u,v) = max_{w\in S_{k−2}}(\pi(k−1,w,u)\times q(v|w,u) \times e(x_k|v)) $

Se evalúa $\pi(4,P,D)$, notando que $x_4 = the$:
$\pi(4,P,D) = max_{w\in S_{2}}(\pi(3,w,P)\times q(D | w, P) \times e(the | D)) $

La expresión $e(the | D)$ es independiente de $w$ por lo que se puede sacar de la expresión de $max$ y reemplazar su valor:
$\pi(4,P,D) = 0.6 \times max_{w\in S_{2}}(\pi(3,w,P)\times q(D | w, P))$

Dado que $S_2 = \{ D, N, V, P \}$ se tiene que calcular los distintos valores de $w$ en 
$\pi(3,w,P)\times q(D | w, P)$ para determinar qué $w$ maximiza esa expresión, sin embargo notar que dado los datos estimados del corpus de entrenamiento se tiene que:

\begin{equation}
  q(D | w, P) =
  \begin{cases}
    0.4 & \text{if $w = N$} \\
    0 & \text{otherwise}
  \end{cases}
\end{equation}

Esto implica que el único $w$ que no hace que la expresión sea nula es $w = N$, por lo que ésta es la que maximiza la expresión. Por lo que la expresión resultante es:

$\pi(4,P,D) = 0.6 \times (\pi(3,N,P)\times q(D | N, P))$

$\pi(4,P,D) = 0.6 \times (0.2 \times 0.4) = 0.048$

### Pregunta 2 (0.5 pts)
Comente  sobre las similitudes o diferencias entre los HMMs, MEMMs y CRFs. Para esto, responda las siguientes preguntas.
* ¿Para qué tipo de tarea sirven? Dé dos ejemplo de este tipo de tarea y descríbalos brevemente.
* ¿Qué modelos usan features? ¿Qué ventajas conlleva esto?
* ¿Cómo maneja cada uno de los modelos las palabras con baja frecuencia en el set de train?
* ¿Qué le permite a los CRF realizar decisiones globales? ¿Qué diferencia con respecto a los MEMMs permite lograr esto? ¿Por qué los HMMs tampoco son capaces de tomar decisiones globales?

**Respuestas**
* Estos modelos sirven para resolver problemas de Tagging, como el Part of Speech (PoS)
o Name Entity Recognition (NER). Dada un documento o texto como input, la tarea de PoS es definir para cada palabra del input
determinar si ésta es un Sustantivo, Adjetivo, Verbo, Adverbio u otro tipo, mientras que la tarea
de NER es determinar qué partes del input pertenecen a algún tipo de entidad (sujeto).
* Los modelos de MEMMs y CRF utilizan features, esto permite tener conocimiento de todas las etiquetas a clasificar, además permite obtener un
manejo especial de las etiquetas a clasificar como actuar según prefijos o sufijos.
* Los modelos HMMs separa las palabras con alto conteo ($\geq5$) con las de bajo conteo y las clasifica utilizando reglas especiales
determinadas según la tarea a resolver. Los modelos MEMMs y CRFs dentro del vector de features pueden contabilizar especialmente éstas etiquetas
de baja frecuencia para agruparlas en tipos más grandes.
* ...

# Convolutional Neural Networks
### Pregunta 3 (1 pt)

Considere la frase $w_{1..7}=$ "El agua moja y el fuego quema" $=[El, agua, moja, y, el, fuego, quema]$.

La siguiente matriz de embeddings, donde la i-ésima fila corresponde al vector de embedding de la i-ésima palabra, ordenadas segun aparecen en la frase. (vectores de largo 2).
\begin{equation}
E = \begin{pmatrix}
2 & 2\\
0 & -2\\
0 & 1\\
-2 & 1\\
-2 & 0\\
2 & -1\\
0 & 2
\end{pmatrix}
\end{equation}

Los siguientes 3 filtros
\begin{equation}
U = \begin{pmatrix}
-1 & -1 & 0\\
1 & 1 & 0\\
0 & 0 & -1\\
-1 & -1 & -1\\
-1 & -1 & 1\\
1 & 0 & -1
\end{pmatrix}
\end{equation}

Y la función de activación
\begin{equation}
tanh = \frac{e^{2x} - 1}{e^{2x} + 1}
\end{equation}

Usando estos paramátros calcule manualmente la representación (vector) resultante de aplicar la operación de convolución (sin padding) + max pooling.


**Respuesta**

Los 3 filtros son tamaño 6, los embeddings son de 2 dimensiones, de esta
forma el tamaño de la ventana es 3. Esto dan los siguientes trigramas, con sus
vectores de embedding unidos asociados:
* "El agua moja" $\to (2,2,0,-2,0,1)$
* "agua moja y" $\to (0,-2,0,1,-2,1)$
* "moja y el" $\to (0,1,-2,1,-2,0)$
* "y el fuego" $\to (-2,1,-2,0,2,-1)$
* "el fuego quema" $\to (-2,0,2,-1,0,2)$

Cada uno de estos vectos se le calculará el producto punto de cada filtro de $U$.
Lo cual nos da los siguientes valores:
* $(2,2,0,-2,0,1) \to ( 3,  2,  1)$
* $(0,-2,0,1,-2,1)  \to    ( 0, -1, -4)$
* $(0,1,-2,1,-2,0)  \to    ( 2,  2, -1)$
* $(-2,1,-2,0,2,-1) \to    (0,  1,  5)$
* $(-2,0,2,-1,0,2)  \to  ( 5,  3, -3)$

Ahora cada valor del resultado se le aplicó la función de activación $tanh$
(se aproximó a 3 cifras significativas):

* $( 3,  2,  1) \to (0.995,0.964,0.761)$
* $( 0, -1, -4) \to (0,-0.762,-0.999)$
* $( 2,  2, -1) \to (0.964,0.964,-0.762)$
* $( 0,  1,  5) \to (0,0.762,1)$
* $( 5,  3, -3) \to (1,0.995,-0.995)$

Finalmente se hace el Pooling, que es escojer el máximo para cada columna
de los resultados de cada filtro:

* (1, 0.964, 1)

# Recurrent Neural Networks
### Pregunta 4 (1 pt)
Sea la siguiente oración: `El perro ladra` representada como una secuencia de vectores $x1,x2,x3$. Donde cada vector corresponde al word embedding de cada palabra, que a la vez es un vector de dos dimensiones
\begin{equation}
\begin{split}
x1 &= Embed(El)    &= [1, 0] \\
x2 &= Embed(perro) &= [-1, 1] \\
x3 &= Embed(ladra) &= [1,1]
\end{split}
\end{equation}

Se tiene una red recurrente Elman: 
\begin{equation}
\begin{split}
\vec{s}_i &= R_{SRNN}\left (\vec{x}_i, \vec{s}_{i-1}\right ) = g \left (\vec{s}_{i-1}W^s + \vec{x}_i W^x + \vec{b}\right ) \\
\vec{y}_i &= O_{SRNN}\left(\vec{s}_i\right) = \vec{s}_i \\
\end{split}
\end{equation}
Con
\begin{equation}
\vec{s}_i, \vec{y}_i \in \mathbb{R}^{d_s}, \quad \vec{x}_i \in \mathbb{R}^{d_x}, \quad W^x \in \mathbb{R}^{d_x \times d_s}, \quad W^s \in \mathbb{R}^{d_s \times d_s}, \quad \vec{b} \in \mathbb{R}^{d_s}
\end{equation}
y donde los vectores de estado $s_i$ son de tres dimensiones, $ds= 3$.

Sea
\begin{equation}
\begin{split}
\vec{s}_0 &= [0,0,0]\\
W^x &= \begin{pmatrix}
0 &  0 & 1\\
1 & -1 & 0
\end{pmatrix} \\
W^s &= \begin{pmatrix}
1 & 0 &  1\\
0 & 1 & -1\\
1 & 1 &  1
\end{pmatrix} \\
\vec{b} &= [0, 0, 0] \\
g(x) &= ReLu(x) = max(0, x)
\end{split}
\end{equation}

<br>

Calcule manualmente los valores de $\vec{s}_1, \vec{s}_2,\vec{s}_3$ y de $\vec{y}_1, \vec{y}_2,\vec{y}_3$.

**Respuesta**

Para cada palabra $\vec{x}_i$ se calula la función $R_{SRNN}(\vec{x}_{i},\vec{s}_{i-1})$ para obtener el estado $\vec{s}_i$,
para este cálculo se utiliza el vector de embedding de ésta palabra.

Luego, se realizan las multiplicaciones de matrices, la suma de éstos vectores resultantes para finalmente
para obtener el estado siguiente utilizando la
función de activación dada $g(x) = max(0,x)$.

Con este estado se utiliza la función $O_{SRRN}$ para obtener el vector $\vec{y}_i$

Dado que se obtiene el estado $\vec{s}_{i}$ se puede calcular el estado siguiente a éste utilizando
$\vec{x}_{i+1}.

Se calcula en primera instancia $R_{SRNN}(\vec{x}_1,\vec{s_0})$ 
donde:
$\vec{x}_1 = Embed(El) = (1,0)$
\begin{equation}
\vec{s}_1 = R_{SRNN}(\vec{x}_1,\vec{s_0}) = ReLU(\vec{s}_{0}W^s + \vec{x}_1 W^x + \vec{b}) \\
= ReLU(
\begin{pmatrix}
0,0,0
\end{pmatrix} 
\begin{pmatrix}
1 & 0 &  1\\
0 & 1 & -1\\
1 & 1 &  1
\end{pmatrix} + 
\begin{pmatrix}
1,0
\end{pmatrix} 
\begin{pmatrix}
0 &  0 & 1\\
1 & -1 & 0
\end{pmatrix} + (0,0,0))  \\ 
\end{equation}
\begin{equation}
= ReLU( (0,0,0) + (0,0,1) + (0,0,0))
\end{equation}
\begin{equation}
= ReLU( (0,0,1)) = (0,0,1)
\end{equation}

Así: $\vec{y}_1 = O_{SRNN}(\vec{s}_1) = (0,0,1)$


---

$\vec{x}_2 = Embed(perro) = (-1,1)$
\begin{equation}
\vec{s}_2 = R_{SRNN}(\vec{x}_2,\vec{s}_1) = ReLU(\vec{s}_{1}W^s + \vec{x}_2 W^x + \vec{b}) \\
= ReLU(
\begin{pmatrix}
0,0,1
\end{pmatrix} 
\begin{pmatrix}
1 & 0 &  1\\
0 & 1 & -1\\
1 & 1 &  1
\end{pmatrix} + 
\begin{pmatrix}
-1,1
\end{pmatrix} 
\begin{pmatrix}
0 &  0 & 1\\
1 & -1 & 0
\end{pmatrix} + (0,0,0))  \\ 
\end{equation}
\begin{equation}
= ReLU( (1,1,1) + (1,-1,-1) + (0,0,0))
\end{equation}
\begin{equation}
= ReLU( (2,0,0)) = (2,0,0)
\end{equation}

Así: $\vec{y}_2 = O_{SRNN}(\vec{s}_2) = (2,0,0)$

---

$\vec{x}_3 = Embed(ladra) = (1,1)$
\begin{equation}
\vec{s}_3 = R_{SRNN}(\vec{x}_3,\vec{s}_2) = ReLU(\vec{s}_2W^s + \vec{x}_3 W^x + \vec{b}) \\
= ReLU(
\begin{pmatrix}
2,0,0
\end{pmatrix} 
\begin{pmatrix}
1 & 0 &  1\\
0 & 1 & -1\\
1 & 1 &  1
\end{pmatrix} + 
\begin{pmatrix}
1,1
\end{pmatrix} 
\begin{pmatrix}
0 &  0 & 1\\
1 & -1 & 0
\end{pmatrix} + (0,0,0))  \\ 
\end{equation}
\begin{equation}
= ReLU( (2,0,2) + (1,-1,1) + (0,0,0))
\end{equation}
\begin{equation}
= ReLU( (3,-1,3)) = (3,0,3)
\end{equation}

Así: $\vec{y}_2 = O_{SRNN}(\vec{s}_2) = (3,0,3)$

### Pregunta 5 (0.5 pts)
¿De qué forma las RNN y las CNN logran aprender representaciones específicas
para la tarea objetivo? Compare la forma en que las RNN y las CNN aprenden 
con los modelos que usan *features* diseñadas manualmente.

**Respuesta**
Aprenden a través de la refinación de sus parámetros a medidas que se va realizando
el cómputo de la minimización de la loss durante las múltiples iteraciones del entrenamiento.
Por lo que al utilizar algún modelo que utiliza features éste sólo necesita leer una única vez
(contar palabras, largo, etc.) el set de entrenamiento para determinar los parámetros internos para
resolver la tarea.

# Redes neuronales con PyTorch
### Pregunta 6 (2 pts)
En esta parte van a tener que implementar una red neuronal Feed Forward.
Además, deberán entrenar el modelo usando uno de los datasets de TorchText.
En la sección de la respuesta hay un esqueleto de lo que deben hacer,
deberán completar los metodos del modelo y la parte asociada al entrenamiento la deben implementar ustedes.
De todas formas, como les mencionamos en las auxiliares, el proceso de entrenamiento es bastante estándar,
así que se pueden guiar en gran medida por los ejemplos que hemos visto y
los que vamos a ver en las próximas auxiliares.

### Bonus (0.5 pts)
Agregue a la arquitectura una capa convolucional,
para esto debe registrar el parametro $U$ en la red y realizar el computo
de la convolución en el metodo forward de la red.


In [45]:
%%capture
# Nos aseguramos que torchtext este en la ultima version
!pip install --upgrade torchtext

In [46]:
# Trabajen con el siguiente dataset
import os
from random import choice
from torchtext.datasets import AG_NEWS

os.makedirs(".data", exist_ok=True)
train_dataset, test_dataset = AG_NEWS(root=".data")

print("\nUn ejemplo aleatorio\n", choice(train_dataset))

120000lines [00:05, 23073.21lines/s]
120000lines [00:08, 13527.96lines/s]
7600lines [00:00, 14283.47lines/s]



Un ejemplo aleatorio
 (1, tensor([  499,  1319, 32717,     5,  3531,    12,  4522,     3,   742, 29573,
           41,  4325,     4,  1447,  9824, 32717,     9,   707,     5,     3,
         3531,    12, 12353,   336,  6662,  4522,     9,  7063,     6,   269,
            5, 49901,  1122, 17623,    30,     3,  1493,     2]))


In [47]:
from pprint import pprint
# Informacion relevante del dataset
vocab = train_dataset.get_vocab()
labels = train_dataset.get_labels()
# El nombre de cada label lo pueden encontrar aqui
# https://pytorch.org/text/datasets.html#ag-news, aunque no es necesario para 
# clasificar
print(labels)
# Un ejemplo convertido a texto
pprint(" ".join(vocab.itos[token] for token in choice(train_dataset)[1]))

{0, 1, 2, 3}
('state street profit drops , job cuts coming boston ( reuters ) - state '
 "street corp . , the world ' s biggest institutional asset manager , on "
 'tuesday said third-quarter profit fell 12 percent amid lower revenue and '
 'higher expenses , sending its shares tumbling .')


In [48]:
# De aca para abajo viene su respuesta, completen las funciones en la red
# y luego entrenen el modelo y evaluenlo usando los dataset que acaban de
# cargar
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class, pad_idx, cnn_dim, hidden_size): # Reemplacen el *args por sus argumento
        super().__init__()
        self.embedding = nn.Embedding(
            vocab_size,
            embed_dim,
            pad_idx)
        self.cnn = nn.Conv1d(1, cnn_dim, 3*embed_dim,stride=embed_dim) # thx aux 4
        self.fc1 = nn.Linear(cnn_dim, hidden_size) # thx aux 3
        self.fc2 = nn.Linear(hidden_size, num_class)
        # Aca deben registrar sus parametros. A lo menos necesitan
        # una capa de embedding y un MLP basico (una capa lineal + softmax)

    def forward(self, batch):
        # (B, N, E) -> (B, E)
        z = self.embedding(batch)
        z = z.view(z.shape[0], 1, -1)
        z = torch.tanh(self.cnn(z))
        z = z.max(dim=-1).values
        z = F.relu(self.fc1(z))
        z = F.relu(self.fc2(z))
        return torch.softmax(z,-1)

In [49]:
# El resto de su respuesta
# Aca deben programar el entrenamiento de la red

device = "cuda" if torch.cuda.is_available() else "cpu"
from itertools import zip_longest

def generate_batch(batch):
    return (
        # En este caso como los labels son numeros, el tensor es de 1 dimension
        # de tamanno batch_size
        torch.tensor([item[0] for item in batch]),

        # En este caso se retorna un tensor de 2 dimensiones, batch_size x N,
        # donde N es mayor largo de los ejemplo en el batch. Aca se realiza
        # padding de los ejemplos mas cortos.
        torch.tensor(
            list(
                zip(
                    *zip_longest(
                        *[item[1] for item in batch], fillvalue=vocab["<pad>"]
                    )))),)

from torch.utils.data import DataLoader

def train_func(train_dataset):

    # Entranamos el modelo
    train_loss = 0
    train_acc = 0
    data = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        collate_fn=generate_batch,
    )
    for i, (cls, text) in enumerate(data):
        optimizer.zero_grad()
        cls, text = cls.to(device), text.to(device)
        output = model(text)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Ajustar el learning rate
    scheduler.step()

    return train_loss / len(train_dataset), train_acc / len(train_dataset)

def test(test_dataset):
    test_loss = 0
    acc = 0
    data = DataLoader(
        test_dataset, batch_size=BATCH_SIZE, collate_fn=generate_batch
    )
    for cls, text in data:
        cls, text = cls.to(device), text.to(device)
        with torch.no_grad():
            output = model(text)
            loss = criterion(output, cls)
            test_loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return test_loss / len(test_dataset), acc / len(test_dataset)

In [ ]:
import time

N_EPOCHS = 5
LEARN_RATE = 4.0
STEP_SIZE = 1
BATCH_SIZE = 32
EMBED_DIM = 100
CNN_DIM = 35
HIDDEN_SIZE = 200


model = CNNClassifier(
    vocab_size=len(vocab),
    embed_dim=EMBED_DIM,
    num_class=len(labels),
    pad_idx=vocab["<pad>"],
    cnn_dim=CNN_DIM,
    hidden_size=HIDDEN_SIZE).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=LEARN_RATE)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, STEP_SIZE)

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(train_dataset)
    valid_loss, valid_acc = test(test_dataset)

    secs = int(time.time() - start_time)
    mins = secs // 60
    secs = secs % 60

    print(
        f"Epoch: {epoch + 1}", f" | time in {mins} minutes, {secs} seconds",
    )
    print(
        f"\tLoss: {train_loss:.4f}(train)\t|"
        f"\tAcc: {train_acc * 100:.1f}%(train)"
    )
    print(
        f"\tLoss: {valid_loss:.4f}(valid)\t|"
        f"\tAcc: {valid_acc * 100:.1f}%(valid)"
    )

Epoch: 1  | time in 0 minutes, 37 seconds
	Loss: 0.0433(train)	|	Acc: 25.0%(train)
	Loss: 0.0434(valid)	|	Acc: 25.0%(valid)
Epoch: 2  | time in 0 minutes, 39 seconds
	Loss: 0.0433(train)	|	Acc: 25.0%(train)
	Loss: 0.0434(valid)	|	Acc: 25.0%(valid)
Epoch: 3  | time in 0 minutes, 39 seconds
	Loss: 0.0433(train)	|	Acc: 25.0%(train)
	Loss: 0.0434(valid)	|	Acc: 25.0%(valid)
Epoch: 4  | time in 0 minutes, 38 seconds
	Loss: 0.0433(train)	|	Acc: 25.0%(train)
	Loss: 0.0434(valid)	|	Acc: 25.0%(valid)
